In [45]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <-
    house_prices$helpers$load_data() %>%
    (house_prices$outliers$remove_outliers) %>%
    (house_prices$missing$fix_all) %>%
    select(-Id)

test_that("should be no NA values except SalePrice column", {
    expect_equal(sum(is.na(combined_dataset %>% select(-SalePrice))), 0)
})

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 14, repr.plot.res=280)
source('main.R')

trans <- tribble(
    ~tran_name, ~tran_defin,
    'log',      function(x) log(x+1),
    'sqrt',     function(x) sqrt(x),
    'invcube',  function(x) x**(1/3)
)

transformation_config <- 
    house_prices$trans$numeric$get_transformation_config(
        dataset=combined_dataset,
        trans=trans
    ) %>% 
    filter(progress_score > 30)

transformation_config 

df <- house_prices$trans$numeric$for_qq_plot(combined_dataset, transformation_config)
df %>%
ggplot() +
geom_qq(aes(sample=normed_value), alpha=0.2) +
facet_wrap(var~tran, ncol=4, scales="free") +
geom_abline(slope=1) +
theme_bw()

var,tran,progress_score,tran_defin
SalePrice,log,91.29809,"function (x) , log(x + 1)"
GrLivArea,log,91.15794,"function (x) , log(x + 1)"
X1stFlrSF,log,90.77166,"function (x) , log(x + 1)"
BsmtUnfSF,sqrt,79.97603,"function (x) , sqrt(x)"
LotArea,invcube,78.43689,"function (x) , x^(1/3)"
TotRmsAbvGrd,log,30.68502,"function (x) , log(x + 1)"


In [26]:
source('main.R')

transformation_config <- house_prices$trans$numeric$get_transformation_config(
    dataset=combined_dataset,
    trans=trans
)

combined_dataset_transformed <-
    house_prices$trans$numeric$apply_transform(combined_dataset, transformation_config)


test_that("should be equal", {
    expect_equal(
        combined_dataset_transformed %>% select(-one_of(transformation_config$var)), 
        combined_dataset %>% select(-one_of(transformation_config$var))
    )
})

test_that("GrLivArea", {
    expect_equal(
        combined_dataset_transformed$GrLivArea, 
        log(combined_dataset$GrLivArea + 1)
    )
})

test_that("BsmtUnfSF", {
    expect_equal(
        combined_dataset_transformed$BsmtUnfSF, 
        sqrt(combined_dataset$BsmtUnfSF)
    )
})